In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import xgboost as xgb

import pickle

In [2]:
# data_path = '../data/'
# test = pd.read_csv(data_path + 'test_fill.csv')
# train = pd.read_csv(data_path + 'train_fill.csv', index_col=0)

In [3]:
# print train.head(0)

In [4]:
data_path = '../data/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
songs = pd.read_csv(data_path + 'songs1.csv')
members = pd.read_csv(data_path + 'members.csv')

song_cols = ['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',\
            'lyricist', 'language', 'song_name', 'song_country', 'song_year']

train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[6:8]))

members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration_date'] = members['expiration_date'].apply(lambda x: int(str(x)[6:8]))
member = members.drop(['registration_year', 'expiration_year'], axis=1)

members_cols = members.columns
train = train.merge(members[members_cols], on='msno', how='left')
test = test.merge(members[members_cols], on='msno', how='left')

train = train.fillna(-1)
test = test.fillna(-1)

In [5]:
#<class 'pandas.core.series.Series'> object
#pandas.core.frame.DataFrame
print train.head()

                                           msno  \
0  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
1  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
2  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
3  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
4  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   

                                        song_id source_system_tab  \
0  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=           explore   
1  bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=        my library   
2  JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=        my library   
3  2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=        my library   
4  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=           explore   

    source_screen_name      source_type  target  song_length genre_ids  \
0              Explore  online-playlist       1     206471.0       359   
1  Local playlist more   local-playlist       1     284584.0      1259   
2  Local playlist more   local-playlist       1     2253

In [6]:
# Preprocess dataset
cols = list(train.columns)
cols.remove('target')

for col in tqdm(cols):
    try:
        if train[col].dtype == 'object':
            train[col] = train[col].apply(str)
            test[col] = test[col].apply(str)

            le = LabelEncoder()
            train_vals = list(train[col].unique())
            test_vals = list(test[col].unique())
            le.fit(train_vals + test_vals)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

            print(col + ': ' + str(len(train_vals)) + ', ' + str(len(test_vals)))
    except:
        print (col, type(train[col]))
        break

  4%|▍         | 1/25 [00:37<15:06, 37.77s/it]

msno: 30755, 25131


  8%|▊         | 2/25 [02:14<25:50, 67.41s/it]

song_id: 359966, 224753


 12%|█▏        | 3/25 [02:34<18:49, 51.35s/it]

...

song_name: 234112, 154708


 52%|█████▏    | 13/25 [07:40<07:04, 35.39s/it]

song_country: 110, 97


100%|██████████| 25/25 [07:56<00:00, 19.04s/it]

gender: 3, 3


In [7]:
print (train.head())
    

    msno  song_id  source_system_tab  source_screen_name  source_type  target  \
0   9176    86884                  2                   8            7       1   
1  19273   260594                  4                   9            5       1   
2  19273   140755                  4                   9            5       1   
3  19273    27577                  4                   9            5       1   
4   9176    38706                  2                   8            7       1   

   song_length  genre_ids  artist_name  composer        ...         bd  \
0     206471.0        308         3785     16654        ...          0   
1     284584.0         98        36868        71        ...         24   
2     225396.0         98        24602     51541        ...         24   
3     255512.0          7        31652     41992        ...         24   
4     187802.0          3         5191      9702        ...          0   

   gender  registered_via  registration_init_time  expiration_date  

In [22]:
X = np.array(train.drop(['target'], axis=1))
y = train['target'].values

X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, \
#     test_size=0.2, random_state=0)
l = int(np.shape(X)[0] * 0.8)
X_train, X_valid, y_train, y_valid = X[:l], X[l:], y[:l], y[l:]


d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid) 
d_test = xgb.DMatrix(X_test)

eval_set = [(X_train, y_train), (X_valid, y_valid)]
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [21]:
print np.shape(X)[0]
print  int(np.shape(X)[0] * 0.8)

7377418
5901934


In [23]:
# Train model, evaluate and make predictions
def model(eta, max_depth):
    params = {}
    params['objective'] = 'binary:logistic'
    params['eta'] = eta #0.7
    params['max_depth'] = max_depth #5
    params['silent'] = 1
    params['eval_metric'] = 'auc'
    params['subsample'] = 1.0 #1.0
    params['min_child_weight'] = 5 #5
    params['colsample_bytree'] = 0.2 # 0.2
    evals_result = {}
    
    model = xgb.train(params, d_train, 105, watchlist, early_stopping_rounds=20, \
        maximize=True, verbose_eval=10, evals_result = evals_result)

    p_test = model.predict(d_test)
    return [p_test, [evals_result['train']['auc'][-1], evals_result['valid']['auc'][-1]]]

In [24]:
etas = [0.01 * (2 ** x) for x in range(1, 8)] # 0.01 to 1.28
subsamples = [0.1 * x for x in range(5, 11)]
min_child_weights = [ x for x in range(2, 10)] 
colsample_bytrees = [0.1 * x for x in range(1, 11)]

In [25]:
p_tests = []
weights = []
for max_depth in range(15, 21):
    for eta in [0.3]:
        p_test, res = model(eta, max_depth)
        with open('%s_%s.pickle' % (eta*10, max_depth),\
                                      'wb')as handle:
            pickle.dump(p_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('%s_%s_res.pickle' % (eta*10, max_depth),\
                                      'wb')as handle:
            pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)
        p_tests += p_test,
        weights += res,

[0]	train-auc:0.668375	valid-auc:0.597764
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.


KeyboardInterrupt: 

In [12]:
weights = [x[1] for x in weights]
print (weights)

[0.793117, 0.795991, 0.798134, 0.79797, 0.799357, 0.799654]


In [13]:
total = np.sum(weights)
print (total)
result = (np.array(p_tests).T * np.array(weights)).T

4.784223


In [14]:
print (np.shape(result))
mat = np.sum(np.matrix(result), axis = 0)
p_test = [x / total for x in (mat.tolist()[0])]
print (np.shape(p_test))

(6, 2556790)
(2556790,)


In [15]:
for x in p_test[:10]:
    print (x)

0.480637092896
0.576250453345
0.149212460832
0.085915018623
0.0474071012508
0.147139892911
0.11923072288
0.686660507123
0.0615600008833
0.759779905435


In [16]:
# with open('p_tests.pickle', 'wb') as handle:
#     pickle.dump(p_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('weights.pickle', 'wb') as handle:
#     pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
print (np.sum(p_test))

1123379.68809


In [18]:
# Prepare submission
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
print(len(ids), len(p_test))
subm.to_csv('../submissions/submission.csv', index=False)

(2556790, 2556790)
